In [ ]:
import numpy as np

In [ ]:
class Dense_Layer:
    def __init__(self, weights, biases, activation="relu"):
        self.weights = np.array(weights, dtype=float)
        self.biases = np.array(biases, dtype=float)
        self.activation = activation

    def forward(self, inputs):
        x = np.array(inputs, dtype=float)
        z = self.weights.dot(x) + self.biases
        a = self._activate(z)
        return z, a

    def _activate(self, z):
        if self.activation == "relu":
            return np.maximum(0, z)
        elif self.activation == "sigmoid":
            return 1.0 / (1.0 + np.exp(-z))
        elif self.activation == "softmax":
            exp_vals = np.exp(z - np.max(z))
            return exp_vals / np.sum(exp_vals)
        else:
            raise ValueError("Unsupported activation: " + str(self.activation))

In [ ]:
class NeuralNetwork:
    def __init__(self):
        self.layers = []

    def add_layer(self, weights, biases, activation="relu"):
        self.layers.append(Dense_Layer(weights, biases, activation))

    def forward_pass(self, inputs, verbose=False, round_digits=4):
        x = np.array(inputs, dtype=float)
        intermediates = []
        for i, layer in enumerate(self.layers, start=1):
            z, a = layer.forward(x)
            intermediates.append({"z": z, "a": a, "activation": layer.activation})
            if verbose:
                print(f"  Layer {i} (activation={layer.activation})")
                print("    Z =", np.round(z, round_digits))
                print("    A =", np.round(a, round_digits))
            x = a
        return intermediates

In [ ]:
# Loss functions
def categorical_cross_entropy(y_pred, y_true):
    eps = 1e-15
    y_pred = np.clip(y_pred, eps, 1 - eps)
    return -np.sum(np.array(y_true) * np.log(y_pred))

In [ ]:
def binary_cross_entropy(y_pred, y_true):
    eps = 1e-15
    y_pred = np.clip(y_pred, eps, 1 - eps)
    return - (y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

In [ ]:
if __name__ == "__main__":
    # -----------------------
    # ARIANE: Problem A (Iris dataset)
    # -----------------------
    print("<Problem A>")
    print("-" * 40)
    X = [5.1, 3.5, 1.4, 0.2]
    target = [0.7, 0.2, 0.1]

    model = NeuralNetwork()
    # First hidden layer
    model.add_layer(
        weights=[[0.2, 0.5, -0.3, 0.1],
                 [-0.2, 0.4, -0.4, 0.3],
                 [0.2, 0.6, -0.1, 0.5]],
        biases=[3.0, -2.1, 0.6],
        activation="relu"
    )
    # Second hidden layer
    model.add_layer(
        weights=[[0.3, -0.5, 0.7],
                 [0.2, -0.6, 0.4]],
        biases=[4.3, 6.4],
        activation="sigmoid"
    )
    # Output layer
    model.add_layer(
        weights=[[0.5, -0.3],
                 [0.8, -0.2],
                 [0.6, -0.4]],
        biases=[-1.5, 2.1, -3.3],
        activation="softmax"
    )

    results = model.forward_pass(X, verbose=True, round_digits=4)
    print("\nHidden Layer 2 (Output):", np.round(results[1]["a"], 4))
    final_output = results[-1]["a"]
    print("Final Output (Softmax):", np.round(final_output, 6))

    classes = ["Iris-setosa", "Iris-versicolor", "Iris-virginica"]
    print("Predicted Class:", classes[int(np.argmax(final_output))])
    loss = categorical_cross_entropy(final_output, target)
    print("Loss:", round(loss, 6))
    print("-" * 40, "\n")

    # -----------------------
    # LEANN: Problem B (Breast Cancer dataset)
    # -----------------------
    print("<Problem B>")
    print("-" * 40)
    Xb = [14.1, 20.3, 0.095]
    target_b = 1.0  # malignant

    model_b = NeuralNetwork()
    # First hidden layer
    model_b.add_layer(
        weights=[[0.5, -0.3, 0.8],
                 [0.2, 0.4, -0.6],
                 [-0.7, 0.9, 0.1]],
        biases=[0.3, -0.5, 0.6],
        activation="relu"
    )
    # Second hidden layer
    model_b.add_layer(
        weights=[[0.6, -0.2, 0.4],
                 [-0.3, 0.5, 0.7]],
        biases=[0.1, -0.8],
        activation="sigmoid"
    )
    # Output layer
    model_b.add_layer(
        weights=[[0.7, -0.5]],
        biases=[0.2],
        activation="sigmoid"
    )

    results_b = model_b.forward_pass(Xb, verbose=True, round_digits=4)
    print("\nHidden Layer 2 (Output):", np.round(results_b[1]["a"], 4))
    final_output_b = results_b[-1]["a"].item()
    print("Final Output (Sigmoid):", round(final_output_b, 6))

    pred_label = 1 if final_output_b >= 0.5 else 0
    print("Predicted Class:", "Malignant (1)" if pred_label == 1 else "Benign (0)")
    loss_b = binary_cross_entropy(final_output_b, target_b)
    print("Loss:", round(loss_b, 6))
    print("-" * 40)